In [10]:
import os
import sys
import boto3
import pandas as pd
from tqdm import tqdm
import numpy as np
import re

from datetime import datetime
import seaborn as sns
from matplotlib import pyplot as plt

In [11]:
tqdm.pandas() 

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [14]:
from src.data import etl

## Configuracion

In [15]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('energizados-incoming-data')
bucket_path = 's3://energizados-incoming-data/' 

In [16]:
DATA_PATH_RAW = bucket_path + 'cr/cnfl/'
DATA_PATH_CLEAN = '../../data/interim/'
var_tramites = ['id','id_tipo_orden','tipo_orden','localizacion','fecha_creacion','fecha_ejecucion','id_cod_resultado','resultado','estado_orden']
var_puntos = ['localizacion','id_provincia','id_canton','id_distrito','id_tipo_medidor','tipo_conexion','area_peligrosa','id_tarifa',
              'id_estado_contrato','tip_instalacion','id_cant_fases','id_ciu','marca_medidor','nic', 'tipo_cliente', 'medidor','ind_fraude', 
              'estado_medidor','id_clase','clasif_contrato','nise','control_manzana','fecha_instalacion']
var_fact = ['localizacion','nise','fecha_referencia','fecha_vence','dia_pago','fecha_medicion','consumo_medido','dias_facturados','cod_facturacion',
            'id_evento_facturacion']

file_name_tramites_clean = 'tramites_fraudes/tramites_clean_{}.parquet'
file_name_tramites_otros_clean = 'tramites_otros/tramites_otros_clean_{}.parquet'
file_name_hist_puntos_clean = 'puntos/hist_puntos_clean_{}.parquet'
file_name_consumo_clean = 'facturacion/consumo_clean_{}.parquet'
file_name_notas_clean = 'notas/notas_clean_{}.parquet'

dtype_dict_puntos = {'LOCALIZACION': str,
 'SISTEMA_COORDENADAS': str,
 'ID_SUCURSAL': str,
 'SUCURSAL': str,
 'ID_PROVINCIA': str,
 'PROVINCIA': str,
 'ID_CANTON': str,
 'CANTON': str,
 'ID_DISTRITO': str,
 'DISTRITO': str,
 'ID_TIPO_MEDIDOR': str,
 'TIPO_MEDIDOR': str,
 'TIPO_CONEXION': str,
 'AREA_PELIGROSA': str,
 'ID_CLASE': str,
 'CLASE': str,
 'TIPO_PENDIENTE': str,
 'ID_TIPO_SISTEMA': str,
 'TIPO_SISTEMA': str,
 'ID_ESTADO_CONTRATO': str,
 'ESTADO_CONTRATO': str,
 'TIP_INSTALACION': str,
 'ID_CANT_FASES': str,
 'CANT_FASES': str,
 'ID_CICLO_LECTURA': str,
 'CICLO_LECTURA': str,
 'ID_CIU': str,
 'CIU': str,
 'MARCA_MEDIDOR': str,
 'NIC': str,
 'NOMBRE_CLIENTE': str,
 'RUTA_SECTOR': str,
 'TIPO_CLIENTE': str,
 'NIVEL_TENSION_PRIMARIA': str,
 'MEDIDOR': str,
 'IND_FRAUDE': str,
 'CONSTANTE_FACTURACION': str,
 'ID_SUBESTACION': str,
 'SUBESTACION': str,
 'ID_CIRCUITO': str,
 'CIRCUITO': str,
 'TRANSFORMADOR': str,
 'CONTROL_MANZANA': str,
 'ESTADO_MEDIDOR': str,
 'COD_CONTROL': str,
 'ID_TARIFA': str,
 'NOM_TARIFA': str,
 'CLASIF_CONTRATO': str,
 'ESTADO_SERVICIO': str,
 'NISE': str}

## Main
---

In [19]:
%%time
lista_anios = [2021]
for y in tqdm(lista_anios,total= len(lista_anios) ) :
    file_name = f'tramites/cnfl_tramites_{y}.csv'
    df,df_t_f = etl.run_etl_historico_ordenes(DATA_PATH_RAW,file_name,var_tramites)
    etl.save_file(df_t_f,DATA_PATH_CLEAN+file_name_tramites_clean.format(y))
    etl.save_file(df,DATA_PATH_CLEAN+file_name_tramites_otros_clean.format(y))
    
    file_name = f'historico_puntos/cnfl_hist_puntos_{y}.csv'
    df = etl.run_etl_datos_hist_puntos(DATA_PATH_RAW,file_name,dtype_dict_puntos)
    etl.save_file(df,DATA_PATH_CLEAN+file_name_hist_puntos_clean.format(y))
    
    file_name = f'facturacion/cnfl_facturacion_{y}.csv'
    df = etl.run_etl_historico_consumo(DATA_PATH_RAW,file_name,var_fact)
    etl.save_file(df,DATA_PATH_CLEAN+file_name_consumo_clean.format(y))
    
    df_notas_grupo = etl.load_notas_group(DATA_PATH_RAW,'notas_lector/notas_group.csv')
    file_name = f'notas_lector/cnfl_notas_lector_{y}.csv'
    df = etl.run_etl_historico_notas_lector(DATA_PATH_RAW,file_name,df_notas_grupo)
    etl.save_file(df,DATA_PATH_CLEAN+file_name_notas_clean.format(y))

  0%|          | 0/1 [00:00<?, ?it/s]/root/aux/cnfl/src/data/etl.py:51: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH_RAW + file_name)
/root/aux/cnfl/src/data/etl.py:101: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH_RAW + file_name)
100%|██████████| 1/1 [19:44<00:00, 1184.08s/it]

CPU times: user 12min 48s, sys: 1min 22s, total: 14min 10s
Wall time: 19min 44s
